In [3]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import csv
import numpy as np
import keras
from keras.layers import Embedding, Reshape, Merge, Dropout, Dense, add, Input, Add, Dot
from keras.models import Sequential, Model, load_model

Using TensorFlow backend.


In [1]:
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]

In [5]:
text = open('train.csv', 'r') 
row = csv.reader(text , delimiter="\n")
count = 0
user = []
movie = []
rating = []
genres_train = []
for r in row:
    if count > 0:
        r = r[0].split(',')
        user.append(r[1])
        movie.append(r[2])
        #genres_train.append(m_genres[int(r[2])-1])
        rating.append(r[3])
    count = 1
text.close()
print(len(user))

899873


In [6]:
user = np.array(user)
movie = np.array(movie)
rating = np.array(rating)
#genres_train = np.array(genres_train)
user=user.astype(np.int)
movie=movie.astype(np.int)
rating=rating.astype(np.int)
#genres_train=genres_train.astype(np.int)

max_userid = np.max(user)
max_movieid = np.max(movie)
print(max_userid)
print(max_movieid)
K_FACTORS = 120

user = user - 1
movie = movie - 1

user, movie, rating = unison_shuffled_copies(user, movie, rating)

6040
3952


In [48]:
#normalize
#means = np.mean(rating)
#std = np.std(rating)
#rating = (rating - means)/std
#print(means)
#print(std)
#print(rating)

3.58171208604
1.11689766115
[ 0.37450872  0.37450872  0.37450872 ...,  0.37450872  0.37450872
 -1.41616564]


In [7]:
from keras import backend as K

def rmse(y_true, y_pred): return K.sqrt( K.mean((y_pred - y_true)**2) )

In [8]:
x = Input(shape=(1,))
y = Input(shape=(1,))
Z = Input(shape=(1,))
embedding_layer_U = Embedding(max_userid, 256 , input_length=1)
embedding_layer_M = Embedding(max_movieid, 256 , input_length=1)
User = embedding_layer_U(x)
#User = Reshape((120,))(User)
User = Dropout(0.5)(Reshape((256,))(User))
Movie = embedding_layer_M(y)
#Movie =Reshape((120,))(Movie)
Movie = Dropout(0.5)(Reshape((256,))(Movie))
R = Dot(axes=1)([User, Movie])

embedding_layer_U_bias = Embedding(max_userid, 1 , input_length=1, embeddings_initializer='zeros')
embedding_layer_M_bias = Embedding(max_movieid, 1 , input_length=1, embeddings_initializer='zeros')
embedding_layer_M_genres = Embedding(4, 1 , input_length=1, embeddings_initializer='zeros')

Bias_U = embedding_layer_U_bias(x)
Bias_U = Reshape((1,))(Bias_U)
Bias_M = embedding_layer_M_bias(y)
Bias_M = Reshape((1,))(Bias_M)

Bias_genres = embedding_layer_M_genres(Z)
Bias_genres = Reshape((1,))(Bias_genres)

Bias = Add()([Bias_U, Bias_M])
Bias = Add()([Bias, Bias_genres])

final = Add()([R, Bias])
model = Model([x, y], R)

In [9]:
model.compile(loss = 'mse', optimizer='adam', metrics=[rmse])
filepath = 'hw5_nobias.hdf5'
callbacks = [EarlyStopping(monitor='val_rmse', patience=20, verbose=1, mode='min'), 
             ModelCheckpoint(filepath, monitor='val_rmse', save_best_only=True, verbose=1, mode='min')]
model.fit([user, movie], rating, epochs=500, batch_size=10000, validation_split=0.05, verbose=1, callbacks=callbacks)


Train on 854879 samples, validate on 44994 samples
Epoch 1/500
854879/854879 [==============================] - 12s - loss: 14.0710 - rmse: 3.7511 - val_loss: 14.0141 - val_rmse: 3.7435
Epoch 2/500
854879/854879 [==============================] - 12s - loss: 13.3743 - rmse: 3.6560 - val_loss: 11.5243 - val_rmse: 3.3947
Epoch 3/500
854879/854879 [==============================] - 12s - loss: 6.7964 - rmse: 2.5523 - val_loss: 2.6895 - val_rmse: 1.6400
Epoch 4/500
854879/854879 [==============================] - 12s - loss: 1.9088 - rmse: 1.3752 - val_loss: 1.2454 - val_rmse: 1.1160
Epoch 5/500
854879/854879 [==============================] - 10s - loss: 1.2756 - rmse: 1.1290 - val_loss: 0.9802 - val_rmse: 0.9900
Epoch 6/500
854879/854879 [==============================] - 10s - loss: 1.1260 - rmse: 1.0611 - val_loss: 0.8957 - val_rmse: 0.9464
Epoch 7/500
854879/854879 [==============================] - 10s - loss: 1.0709 - rmse: 1.0348 - val_loss: 0.8617 - val_rmse: 0.9282
Epoch 8/500
85

854879/854879 [==============================] - 9s - loss: 0.8942 - rmse: 0.9456 - val_loss: 0.7479 - val_rmse: 0.8648
Epoch 28/500
854879/854879 [==============================] - 9s - loss: 0.8917 - rmse: 0.9442 - val_loss: 0.7460 - val_rmse: 0.8637
Epoch 29/500
854879/854879 [==============================] - 9s - loss: 0.8882 - rmse: 0.9424 - val_loss: 0.7444 - val_rmse: 0.8628
Epoch 30/500
854879/854879 [==============================] - 9s - loss: 0.8854 - rmse: 0.9409 - val_loss: 0.7425 - val_rmse: 0.8617
Epoch 31/500
854879/854879 [==============================] - 9s - loss: 0.8832 - rmse: 0.9398 - val_loss: 0.7419 - val_rmse: 0.8613
Epoch 32/500
854879/854879 [==============================] - 9s - loss: 0.8826 - rmse: 0.9394 - val_loss: 0.7415 - val_rmse: 0.8611
Epoch 33/500
854879/854879 [==============================] - 9s - loss: 0.8797 - rmse: 0.9379 - val_loss: 0.7391 - val_rmse: 0.8597
Epoch 34/500
854879/854879 [==============================] - 9s - loss: 0.8780 - 

854879/854879 [==============================] - 9s - loss: 0.8484 - rmse: 0.9211 - val_loss: 0.7247 - val_rmse: 0.8512
Epoch 56/500
854879/854879 [==============================] - 9s - loss: 0.8480 - rmse: 0.9208 - val_loss: 0.7240 - val_rmse: 0.8509
Epoch 57/500
854879/854879 [==============================] - 9s - loss: 0.8458 - rmse: 0.9197 - val_loss: 0.7245 - val_rmse: 0.8512
Epoch 58/500
854879/854879 [==============================] - 9s - loss: 0.8470 - rmse: 0.9203 - val_loss: 0.7236 - val_rmse: 0.8506
Epoch 59/500
854879/854879 [==============================] - 9s - loss: 0.8464 - rmse: 0.9200 - val_loss: 0.7251 - val_rmse: 0.8515
Epoch 60/500
854879/854879 [==============================] - 9s - loss: 0.8447 - rmse: 0.9191 - val_loss: 0.7237 - val_rmse: 0.8507
Epoch 61/500
854879/854879 [==============================] - 9s - loss: 0.8441 - rmse: 0.9187 - val_loss: 0.7233 - val_rmse: 0.8504
Epoch 62/500
854879/854879 [==============================] - 9s - loss: 0.8441 - 

854879/854879 [==============================] - 8s - loss: 0.8290 - rmse: 0.9105 - val_loss: 0.7200 - val_rmse: 0.8485
Epoch 85/500
854879/854879 [==============================] - 8s - loss: 0.8313 - rmse: 0.9117 - val_loss: 0.7205 - val_rmse: 0.8488
Epoch 86/500
854879/854879 [==============================] - 8s - loss: 0.8297 - rmse: 0.9109 - val_loss: 0.7200 - val_rmse: 0.8485
Epoch 87/500
854879/854879 [==============================] - 8s - loss: 0.8289 - rmse: 0.9104 - val_loss: 0.7205 - val_rmse: 0.8488
Epoch 88/500
854879/854879 [==============================] - 8s - loss: 0.8273 - rmse: 0.9095 - val_loss: 0.7198 - val_rmse: 0.8484
Epoch 89/500
854879/854879 [==============================] - 8s - loss: 0.8273 - rmse: 0.9096 - val_loss: 0.7210 - val_rmse: 0.8491
Epoch 90/500
854879/854879 [==============================] - 8s - loss: 0.8284 - rmse: 0.9102 - val_loss: 0.7200 - val_rmse: 0.8485
Epoch 91/500
854879/854879 [==============================] - 8s - loss: 0.8264 - 

854879/854879 [==============================] - 8s - loss: 0.8195 - rmse: 0.9052 - val_loss: 0.7187 - val_rmse: 0.8477
Epoch 116/500
854879/854879 [==============================] - 8s - loss: 0.8214 - rmse: 0.9063 - val_loss: 0.7187 - val_rmse: 0.8478
Epoch 117/500
854879/854879 [==============================] - 261s - loss: 0.8190 - rmse: 0.9049 - val_loss: 0.7196 - val_rmse: 0.8483
Epoch 118/500
854879/854879 [==============================] - 11s - loss: 0.8169 - rmse: 0.9038 - val_loss: 0.7182 - val_rmse: 0.8474
Epoch 119/500
854879/854879 [==============================] - 11s - loss: 0.8172 - rmse: 0.9040 - val_loss: 0.7192 - val_rmse: 0.8481
Epoch 120/500
854879/854879 [==============================] - 9s - loss: 0.8201 - rmse: 0.9056 - val_loss: 0.7187 - val_rmse: 0.8478
Epoch 121/500
854879/854879 [==============================] - 10s - loss: 0.8190 - rmse: 0.9050 - val_loss: 0.7188 - val_rmse: 0.8478
Epoch 122/500
854879/854879 [==============================] - 11s - lo

854879/854879 [==============================] - 9s - loss: 0.8146 - rmse: 0.9025 - val_loss: 0.7169 - val_rmse: 0.8467
Epoch 146/500
854879/854879 [==============================] - 9s - loss: 0.8146 - rmse: 0.9026 - val_loss: 0.7178 - val_rmse: 0.8472
Epoch 147/500
854879/854879 [==============================] - 9s - loss: 0.8147 - rmse: 0.9026 - val_loss: 0.7176 - val_rmse: 0.8471
Epoch 148/500
854879/854879 [==============================] - 9s - loss: 0.8137 - rmse: 0.9020 - val_loss: 0.7192 - val_rmse: 0.8480
Epoch 149/500
854879/854879 [==============================] - 9s - loss: 0.8151 - rmse: 0.9028 - val_loss: 0.7181 - val_rmse: 0.8474
Epoch 150/500
854879/854879 [==============================] - 9s - loss: 0.8157 - rmse: 0.9031 - val_loss: 0.7192 - val_rmse: 0.8481
Epoch 151/500
854879/854879 [==============================] - 9s - loss: 0.8131 - rmse: 0.9017 - val_loss: 0.7192 - val_rmse: 0.8481
Epoch 152/500
854879/854879 [==============================] - 9s - loss: 0.

In [10]:
text = open('test.csv', 'r') 
row = csv.reader(text , delimiter="\n")
count = 0
user_pred = []
movie_pred = []
for r in row:
    if count > 0:
        r = r[0].split(',')
        user_pred.append(r[1])
        movie_pred.append(r[2])
    count = 1
text.close()
print(len(user_pred))

100336


In [11]:
text = open('test.csv', 'r') 
row = csv.reader(text , delimiter="\n")
count = 0
user_pred = []
movie_pred = []
genres_pred = []
for r in row:
    if count > 0:
        r = r[0].split(',')
        user_pred.append(r[1])
        movie_pred.append(r[2])
        #genres_pred.append(m_genres[int(r[2])-1])
    count = 1
text.close()
print(len(user_pred))

100336


In [12]:
user_pred = np.array(user_pred)
movie_pred = np.array(movie_pred)

user_pred=user_pred.astype(np.int)
movie_pred=movie_pred.astype(np.int)

#genres_pred = np.array(genres_pred)
#genres_pred=genres_pred.astype(np.int)

max_user_pred = np.max(user_pred)
max_movie_pred = np.max(movie_pred)
print(max_user_pred)
print(max_movie_pred)
#K_FACTORS = 120

user_pred = user_pred - 1
movie_pred = movie_pred - 1


6040
3952


In [13]:
#from keras.models import load_model
model_pred = load_model('hw5_nobias.hdf5', custom_objects={'rmse': rmse})
predict = model_pred.predict([user_pred, movie_pred])
#predict = []
#for i in range(len(user_pred)):
#    predict.append(model.rate(user_pred[i], movie_pred[i]))
print(len(predict))

100336


In [156]:
#normalize
#predict = predict * std + means

In [14]:
predict = np.clip(predict, 1, 5)

In [15]:
ans = []
for i in range(len(predict)):
    ans.append([str(i+1)])
    #a = float(round(predict[i][0]))
    a = float(predict[i][0])
    ans[i].append(a)

filename = 'hw5_nobias.csv'
text = open(filename, "w+")
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(['TestDataID','Rating'])
for i in range(len(ans)):
    s.writerow(ans[i]) 
text.close()